# =========================
# Load libraries
# =========================

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras import callbacks, layers, models
import keras_metrics as km
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split


# =========================
# Load data
# =========================

In [ ]:
# Load gold DATA
data = pd.read_csv("../../../data/processed/stanford.csv")

train_data, test_data, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2
)

# =========================
# Prepare data
# =========================

In [ ]:
number_of_most_frequency_words = 20000
# Create unique index for every word and fit to training data
tokenizer = Tokenizer(num_words=number_of_most_frequency_words)
tokenizer.fit_on_texts(train_data)

# Turn each tweet into a sequence of integers of equal length
sequences = tokenizer.texts_to_sequences(train_data)
train_corpus_embeddings = pad_sequences(sequences)

# Print the number of unique words found in the data set (not the limit placed
# on the tokenizer), use this as feedback to the num_words arg of Tokenizer().
print("Found %d unique words." % len(tokenizer.word_index))

# =========================
# Split data
# =========================

In [ ]:
# Randomly shuffle data
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_corpus_embeddings = train_corpus_embeddings[indices]
train_labels = train_labels.values[indices]

# Split into training and validation data (approximately 80:20)
x_train = train_corpus_embeddings[:10410]
y_train = train_labels[:10410]
x_val   = train_corpus_embeddings[10410:]
y_val   = train_labels[10410:]

# =========================
# Callbacks
# =========================

In [ ]:
callbacks = [ callbacks.TensorBoard(
    log_dir='log_lstm_embedding_wiki',
    histogram_freq=1,
    embeddings_freq = 1,
    embeddings_data = x_train[:100],
    ),
    callbacks.EarlyStopping(
    monitor='acc',
    patience=1,
    ),
    callbacks.ModelCheckpoint(
    filepath='my_model.h5',
    monitor='val_loss',
    save_best_only=True,
        ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
         factor=0.1,
        patience=10,
        ),
]

# =========================
# Build model
# =========================

In [ ]:
# Add Embedding layer
# The final sigmoid layer outputs probability values between [0, 1]
model = models.Sequential()
model.add(layers.Embedding(number_of_most_frequency_words, 8, input_length=train_corpus_embeddings.shape[1]))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(200))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))

# =========================
# Train model
# =========================

In [ ]:
# As the model outputs probabilities, binary crossentropy is the best loss
# metric as it measures the distance between probability distributions
model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=[km.binary_precision(), km.binary_recall()],
)

history = model.fit(
    x_train, y_train, epochs=20, batch_size=512, callbacks=callbacks, validation_data=(x_val, y_val)
)

# Prep history dictionary
precision = history.history["precision"]
val_precision = history.history["val_precision"]
recall = history.history["recall"]
val_recall = history.history["val_recall"]
epochs = range(1, len(precision) + 1)

# Plot the training and validation precision
plt.plot(epochs, precision, "bo", label="Training precision")
plt.plot(epochs, val_precision, "b", label="Validation precision")
plt.title("Training and validation Precision")
plt.xlabel("Epochs")
plt.ylabel("Precision")
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.clf()
plt.plot(epochs, recall, "bo", label="Training recall")
plt.plot(epochs, val_recall, "b", label="Validation recall")
plt.title("Training and validation recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()
plt.show()


# =========================
# Evaluate on test data
# =========================

In [ ]:
# DO NOT retrain the tokenizer. Use the argument oov_token=True to reserve a
# token for unkown words. See https://bit.ly/2lNh15g

# Prepare data
# Ensure sequences are padded to the same length as training data
x_sequences = tokenizer.texts_to_sequences(test_data)
x_test = pad_sequences(x_sequences, train_corpus_embeddings.shape[1])

# Prepare labels, transform to binary and float32
y_test = test_labels.values

# Print results as ['precision', 'recall'] check names with model.metrics_names
model.evaluate(x_test, y_test)[1:]


# Print results as ['precision', 'recall'] check names with model.metrics_names

# =========================
# F1 Score
# =========================

In [ ]:
y_pred = model.predict(x_test, batch_size=32)
y_pred_roc = y_pred
y_pred = y_pred > 0.5
y_pred = y_pred.flatten()
y_pred = y_pred.astype(int)

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("Marco F1:%f" % f1_score(y_test, y_pred, average="macro"))
print("Micro F1:%f" % f1_score(y_test, y_pred, average="micro"))
print("Weighted F1:%f" % f1_score(y_test, y_pred, average="weighted"))


# =========================
# ROC - AUC
# =========================
    

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_roc)
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')